In [1]:
# Querying Yahoo Finance Data

In [2]:
import requests
import pandas as pd
import numpy as np

In [100]:
# First read with no date parameters to capture dates
# Subsequently, read all contracts for the dates found

In [101]:
## FIGURE OUT HOW TO STREAM AND RECORD THIS DATA DAILY

In [129]:
# Captures TODAY only

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

req = requests.get('http://query1.finance.yahoo.com/v7/finance/options/SPY', headers = headers)

In [156]:
# Collect Dates:
dates = pd.read_json(req.content.decode())

In [157]:
dates = dates.optionChain[1][0]['expirationDates']

In [174]:
all_exp = {}
for exp_date in dates:
    all_exp[exp_date] = pd.read_json(requests.get('http://query1.finance.yahoo.com/v7/finance/options/SPY' + '?date=' + str(exp_date), headers = headers).content.decode())

In [191]:
for each in all_exp:
    all_exp[each] = pd.DataFrame.from_dict(all_exp[each].optionChain.result[0], orient='index')

In [248]:
# All Expiration Dates
all_exp[1640563200][0]['expirationDates']

# Current Expiration Date
all_exp[1640563200].loc['options'][0][0]['expirationDate']

# All Puts and Calls in order (Calls & Puts: Low to High Price)
all_exp[1640563200].loc['options'][0][0]['calls']
all_exp[1640563200].loc['options'][0][0]['puts']

# Number of different price contracts
len(all_exp[1640563200].loc['options'][0][0]['puts'])


# Strikes for a Current Expiration Date
all_exp[1640563200].loc['strikes'][0]



[235.0,
 240.0,
 245.0,
 250.0,
 255.0,
 260.0,
 265.0,
 270.0,
 275.0,
 280.0,
 285.0,
 290.0,
 295.0,
 300.0,
 305.0,
 310.0,
 315.0,
 320.0,
 325.0,
 330.0,
 335.0,
 340.0,
 345.0,
 350.0,
 355.0,
 360.0,
 365.0,
 370.0,
 375.0,
 380.0,
 385.0,
 390.0,
 395.0,
 400.0,
 404.0,
 405.0,
 406.0,
 407.0,
 408.0,
 410.0,
 412.0,
 413.0,
 414.0,
 415.0,
 416.0,
 418.0,
 419.0,
 420.0,
 421.0,
 422.0,
 423.0,
 424.0,
 425.0,
 426.0,
 427.0,
 428.0,
 429.0,
 430.0,
 431.0,
 432.0,
 433.0,
 434.0,
 435.0,
 436.0,
 438.0,
 440.0,
 442.0,
 444.0,
 445.0,
 446.0,
 448.0,
 450.0,
 452.0,
 454.0,
 455.0,
 456.0,
 458.0,
 460.0,
 462.0,
 464.0,
 465.0,
 466.0,
 468.0,
 470.0,
 472.0,
 474.0,
 475.0,
 476.0,
 477.0,
 478.0,
 479.0,
 480.0,
 481.0,
 482.0,
 484.0,
 485.0,
 486.0,
 488.0,
 490.0,
 492.0,
 494.0,
 495.0,
 496.0,
 497.0,
 498.0,
 499.0,
 500.0,
 501.0,
 502.0,
 503.0,
 504.0,
 505.0,
 506.0,
 508.0,
 510.0,
 512.0,
 514.0,
 520.0,
 530.0,
 540.0,
 550.0,
 560.0]

In [249]:
len(all_exp[1640563200].loc['options'][0][0]['puts'])

105

In [1]:
all_exp[1640563200]

NameError: name 'all_exp' is not defined

In [254]:
all_exp[1640563200].loc['options'][0][0]['puts'][104]


{'contractSymbol': 'SPY211227P00540000',
 'strike': 540.0,
 'currency': 'USD',
 'lastPrice': 75.07,
 'change': 0.0,
 'percentChange': 0.0,
 'openInterest': 0,
 'bid': 76.77,
 'ask': 77.32,
 'contractSize': 'REGULAR',
 'expiration': 1640563200,
 'lastTradeDate': 1637689660,
 'impliedVolatility': 0.5117236328125,
 'inTheMoney': True}

In [234]:
# Quite Information Accessible for each element
all_exp[1640563200][0][4].keys()

dict_keys(['language', 'region', 'quoteType', 'quoteSourceName', 'triggerable', 'currency', 'exchange', 'longName', 'messageBoardId', 'exchangeTimezoneName', 'exchangeTimezoneShortName', 'gmtOffSetMilliseconds', 'market', 'esgPopulated', 'firstTradeDateMilliseconds', 'priceHint', 'postMarketChangePercent', 'postMarketTime', 'postMarketPrice', 'postMarketChange', 'regularMarketChange', 'regularMarketChangePercent', 'regularMarketTime', 'regularMarketPrice', 'regularMarketDayHigh', 'regularMarketDayRange', 'regularMarketDayLow', 'regularMarketVolume', 'regularMarketPreviousClose', 'bid', 'ask', 'bidSize', 'askSize', 'fullExchangeName', 'financialCurrency', 'regularMarketOpen', 'averageDailyVolume3Month', 'averageDailyVolume10Day', 'fiftyTwoWeekLowChange', 'fiftyTwoWeekLowChangePercent', 'fiftyTwoWeekRange', 'fiftyTwoWeekHighChange', 'fiftyTwoWeekHighChangePercent', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'trailingAnnualDividendRate', 'trailingPE', 'trailingAnnualDividendYield', 'ytdReturn

In [135]:
df = pd.read_json(dates)

In [189]:
newdf = pd.DataFrame.from_dict(df.optionChain.result[0], orient='index')

In [190]:
newdf

,0
underlyingSymbol,SPY
expirationDates,"[1640131200, 1640217600, 1640563200, 164073600..."
strikes,"[235.0, 240.0, 245.0, 250.0, 255.0, 260.0, 265..."
hasMiniOptions,False
quote,"{'language': 'en-US', 'region': 'US', 'quoteTy..."
options,"[{'expirationDate': 1640131200, 'hasMiniOption..."


In [105]:
# Parse and Track records
req.headers['date']

'Tue, 21 Dec 2021 19:56:55 GMT'

In [128]:
len(newdf.loc['options'][0][0]['puts'])

# Selects a specific contract
newdf.loc['options'][0][0]['calls'][134]


{'contractSymbol': 'SPY211231C00438000',
 'strike': 438.0,
 'currency': 'USD',
 'lastPrice': 21.63,
 'change': -2.4100018,
 'percentChange': -10.024965,
 'volume': 2,
 'openInterest': 15,
 'bid': 24.74,
 'ask': 24.85,
 'contractSize': 'REGULAR',
 'expiration': 1640908800,
 'lastTradeDate': 1640102871,
 'impliedVolatility': 0.16724465576171801,
 'inTheMoney': True}

In [126]:
newdf.loc['expirationDates'][0]
newdf.loc['strikes'][0]

[200.0,
 205.0,
 210.0,
 215.0,
 220.0,
 225.0,
 230.0,
 235.0,
 240.0,
 245.0,
 250.0,
 255.0,
 260.0,
 265.0,
 270.0,
 275.0,
 280.0,
 285.0,
 290.0,
 295.0,
 300.0,
 305.0,
 310.0,
 315.0,
 320.0,
 325.0,
 329.0,
 330.0,
 331.0,
 332.0,
 333.0,
 334.0,
 335.0,
 336.0,
 337.0,
 338.0,
 339.0,
 340.0,
 341.0,
 342.0,
 343.0,
 344.0,
 345.0,
 346.0,
 347.0,
 348.0,
 349.0,
 350.0,
 351.0,
 352.0,
 353.0,
 354.0,
 355.0,
 356.0,
 357.0,
 358.0,
 359.0,
 360.0,
 361.0,
 362.0,
 363.0,
 364.0,
 365.0,
 366.0,
 367.0,
 368.0,
 369.0,
 370.0,
 371.0,
 372.0,
 373.0,
 374.0,
 375.0,
 376.0,
 377.0,
 378.0,
 379.0,
 380.0,
 381.0,
 382.0,
 383.0,
 384.0,
 385.0,
 386.0,
 387.0,
 388.0,
 389.0,
 390.0,
 391.0,
 392.0,
 393.0,
 394.0,
 395.0,
 396.0,
 397.0,
 398.0,
 399.0,
 400.0,
 401.0,
 402.0,
 403.0,
 404.0,
 405.0,
 406.0,
 407.0,
 408.0,
 409.0,
 410.0,
 411.0,
 412.0,
 413.0,
 414.0,
 415.0,
 416.0,
 417.0,
 418.0,
 419.0,
 420.0,
 421.0,
 422.0,
 423.0,
 424.0,
 425.0,
 426.0,
 427.0,
